In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, GRU, Dense, Concatenate, TimeDistributed, RepeatVector
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import Adam, SGD, RMSprop

# 데이터 불러오기
df = pd.read_csv('NVspinData_None_-1_230807.csv')

# 데이터셋 분리: train_set 8 : test_set 2
train_df, test_df = train_test_split(df, shuffle=True, test_size=0.2)
# train_set을 다시 8:2로 나눠서 train_set과 validation_set을 만듦
train_df, val_df = train_test_split(train_df, shuffle=True, test_size=0.2)
# random_state=42

# 모든 시퀀스의 길이 중에서 최대 길이를 구하기
all_sequences = [eval(seq) for seq in df['combination'].values]
max_seq_length = max([len(seq) for seq in all_sequences])

# 각 데이터셋에서 theta, phi, sequence 추출하고 reshape 적용
theta_train = train_df['Theta'].values.reshape(-1, 1)
phi_train = train_df['Phi'].values.reshape(-1, 1)
sequence_train = pad_sequences(train_df['combination'].apply(eval).tolist(), maxlen=max_seq_length, padding='pre')

theta_val = val_df['Theta'].values.reshape(-1, 1)
phi_val = val_df['Phi'].values.reshape(-1, 1)
sequence_val = pad_sequences(val_df['combination'].apply(eval).tolist(), maxlen=max_seq_length, padding='pre')

theta_test = test_df['Theta'].values.reshape(-1, 1)
phi_test = test_df['Phi'].values.reshape(-1, 1)
sequence_test = pad_sequences(test_df['combination'].apply(eval).tolist(), maxlen=max_seq_length, padding='pre')


In [2]:

# 모델 정의
theta_input = Input(shape=(1,), name='theta_input')
phi_input = Input(shape=(1,), name='phi_input')

# theta와 phi를 Concatenate
merged = Concatenate()([theta_input, phi_input])

# 시퀀스를 예측하기 위한 GRU 레이어
repeated_vector = RepeatVector(max_seq_length)(merged)

gru_layer = GRU(64, return_sequences=True, name='gru_layer')(repeated_vector)

output = TimeDistributed(Dense(5, activation='softmax'), name='output_layer')(gru_layer)

model = Model(inputs=[theta_input, phi_input], outputs=output)

# 컴파일 및 훈련
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit([theta_train, phi_train], np.expand_dims(sequence_train, -1), 
                    validation_data=([theta_val, phi_val], np.expand_dims(sequence_val, -1)), epochs=50, batch_size=64)

# 검증
loss, accuracy = model.evaluate([theta_test, phi_test], np.expand_dims(sequence_test, -1))
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")


Epoch 1/50
124/124 [==============================] - 3s 15ms/step - loss: 1.1552 - accuracy: 0.5319 - val_loss: 0.8755 - val_accuracy: 0.6027
Epoch 2/50
124/124 [==============================] - 2s 14ms/step - loss: 0.8502 - accuracy: 0.6010 - val_loss: 0.8386 - val_accuracy: 0.6055
Epoch 3/50
124/124 [==============================] - 2s 13ms/step - loss: 0.8312 - accuracy: 0.6033 - val_loss: 0.8282 - val_accuracy: 0.6066
Epoch 4/50
124/124 [==============================] - 2s 14ms/step - loss: 0.8214 - accuracy: 0.6062 - val_loss: 0.8155 - val_accuracy: 0.6081
Epoch 5/50
124/124 [==============================] - 2s 13ms/step - loss: 0.8134 - accuracy: 0.6086 - val_loss: 0.8034 - val_accuracy: 0.6098
Epoch 6/50
124/124 [==============================] - 2s 13ms/step - loss: 0.8044 - accuracy: 0.6165 - val_loss: 0.7960 - val_accuracy: 0.6248
Epoch 7/50
124/124 [==============================] - 2s 13ms/step - loss: 0.8006 - accuracy: 0.6201 - val_loss: 0.7930 - val_accuracy: 0.6252

In [3]:
# 결과를 저장할 디렉토리 생성
models_dir = 'saved_models'
if not os.path.exists(models_dir):
    os.makedirs(models_dir)
    
# 모델 저장
model.save(os.path.join(models_dir, "GRU_model.h5"))

# # 모델 불러오기
# from tensorflow.keras.models import load_model
# loaded_model = load_model("GRU_model.h5")

c:\Python310\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [14]:
# 테스트 데이터셋에서 10개의 샘플을 무작위로 선택
indices = np.random.choice(len(theta_test), 10)

theta_samples = np.array(theta_test)[indices]
phi_samples = np.array(phi_test)[indices]
sequence_samples = np.array(sequence_test)[indices]

# 모델을 사용하여 예측 수행
predicted_sequences = model.predict([theta_samples, phi_samples])

# 가장 확률이 높은 클래스의 인덱스를 선택
predicted_sequences = np.argmax(predicted_sequences, axis=-1)

# 결과를 DataFrame으로 변환
df_results = pd.DataFrame({
    'Theta': theta_samples.ravel(),
    'Phi': phi_samples.ravel(),
    'Actual Sequence': [list(seq) for seq in sequence_samples],
    'Predicted Sequence': [list(seq) for seq in predicted_sequences]
})

# 결과를 저장할 디렉토리 생성
results_dir = 'samle_test_GRU'
if not os.path.exists(results_dir):
    os.makedirs(results_dir)

# 결과를 CSV 파일로 저장
df_results.to_csv(os.path.join(results_dir, 'GRU_results.csv'), index=False)

print("Results saved to GRU_results.csv")


1/1 [==============================] - 0s 310ms/step
Results saved to GRU_results.csv


In [15]:

from kerastuner.tuners import BayesianOptimization

def build_model(hp):
    theta_input = Input(shape=(1,), name='theta_input')
    phi_input = Input(shape=(1,), name='phi_input')

    merged = Concatenate()([theta_input, phi_input])

    repeated_vector = RepeatVector(max_seq_length)(merged)
    
    gru_layer = GRU(hp.Int('gru_units', min_value=32, max_value=256, step=32),
                      return_sequences=True, name='gru_layer')(repeated_vector)
    
    output = TimeDistributed(Dense(hp.Int('dense_units', min_value=5, max_value=50, step=5),
                                   activation='softmax'), name='output_layer')(gru_layer)

    model = Model(inputs=[theta_input, phi_input], outputs=output)
    
    # 컴파일 설정
    optimizer_choice = hp.Choice('optimizer', ['adam', 'sgd', 'rmsprop'])
    lr = hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='log')
    
    if optimizer_choice == 'adam':
        optimizer = Adam(learning_rate=lr)
    elif optimizer_choice == 'sgd':
        optimizer = SGD(learning_rate=lr)
    else:
        optimizer = RMSprop(learning_rate=lr)
    
    model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    
    return model

tuner = BayesianOptimization(
    build_model,
    objective='val_accuracy',
    max_trials=10,
    executions_per_trial=1,
    directory='gru_tuning',
    project_name='GRU_model_tuning'
)

# 하이퍼파라미터 검색
tuner.search([theta_train, phi_train], np.expand_dims(sequence_train, -1),
             validation_data=([theta_val, phi_val], np.expand_dims(sequence_val, -1)),
             epochs=50,
             batch_size=64)


INFO:tensorflow:Reloading Tuner from gru_tuning\GRU_model_tuning\tuner0.json


INFO:tensorflow:Oracle triggered exit


In [16]:

# 최상의 하이퍼파라미터 출력
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print(f"""
The hyperparameter search is complete. 
The optimal number of units in the GRU layer is {best_hps.get('gru_units')}.
The optimal learning rate for the optimizer is {best_hps.get('learning_rate')}.
The optimal optimizer is {best_hps.get('optimizer')}.
The optimal number of units in the Dense layer is {best_hps.get('dense_units')}.
""")



The hyperparameter search is complete. 
The optimal number of units in the GRU layer is 160.
The optimal learning rate for the optimizer is 0.0017224708407553916.
The optimal optimizer is adam.
The optimal number of units in the Dense layer is 40.



In [17]:

# 결과를 저장할 디렉토리 생성
results_dir = 'tuned_GRU_models_results'
if not os.path.exists(results_dir):
    os.makedirs(results_dir)

# 최상의 하이퍼파라미터 조합 가져오기
best_hps = tuner.get_best_hyperparameters(num_trials=5)

# 각 하이퍼파라미터 조합에 대한 학습 곡선을 그릴 예정
for idx, hp in enumerate(best_hps):
    # Hyperparameters from the trial
    optimizer = hp['optimizer']
    learning_rate = hp['learning_rate']
    gru_units = hp['gru_units']

    print(f"Running with hyperparameters: {hp.values}")
    model = build_model(hp)
    history = model.fit([theta_train, phi_train], np.expand_dims(sequence_train, -1), 
                        validation_data=([theta_val, phi_val], np.expand_dims(sequence_val, -1)), 
                        epochs=50, batch_size=64)  # 고정된 배치 크기 사용
    
    # Plot the loss
    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title(f'Trial {idx+1} - Loss (Optimizer: {optimizer}, LR: {round(learning_rate, 2)}, Units: {gru_units})')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()

    # Plot the accuracy
    plt.subplot(1, 2, 2)
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title(f'Trial {idx+1} - Accuracy (Optimizer: {optimizer}, LR: {round(learning_rate, 2)}, Units: {gru_units})')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.tight_layout()
    
    # 결과를 PNG 파일로 저장
    plt.savefig(os.path.join(results_dir, f'trial_{idx+1}_results.png'))
    plt.close()  # 현재 그린 그래프를 닫아 새로운 그래프를 그릴 준비

Running with hyperparameters: {'gru_units': 160, 'dense_units': 40, 'optimizer': 'adam', 'learning_rate': 0.0017224708407553916}
Epoch 1/50
124/124 [==============================] - 12s 85ms/step - loss: 1.0947 - accuracy: 0.5736 - val_loss: 0.8598 - val_accuracy: 0.6007
Epoch 2/50
124/124 [==============================] - 14s 114ms/step - loss: 0.8476 - accuracy: 0.6000 - val_loss: 0.8191 - val_accuracy: 0.6071
Epoch 3/50
124/124 [==============================] - 15s 122ms/step - loss: 0.8277 - accuracy: 0.6023 - val_loss: 0.8086 - val_accuracy: 0.6093
Epoch 4/50
124/124 [==============================] - 21s 167ms/step - loss: 0.8226 - accuracy: 0.6039 - val_loss: 0.8070 - val_accuracy: 0.6118
Epoch 5/50
124/124 [==============================] - 21s 168ms/step - loss: 0.8074 - accuracy: 0.6082 - val_loss: 0.7907 - val_accuracy: 0.6170
Epoch 6/50
124/124 [==============================] - 21s 168ms/step - loss: 0.8011 - accuracy: 0.6119 - val_loss: 0.8091 - val_accuracy: 0.6145
Ep